In [36]:
import cortex
import nibabel
import numpy as np
import matplotlib
import glob
import os
from mapdata.nsd_mapdata import nsd_mapdata
from mapdata.nsd_datalocation import nsd_datalocation

<a id='back_to_top'></a>

This notebook is not intended to be run sequentially beginning to end. \
It contains various examples of pycortex or pycortex-related tasks, such as getting your \
functional data to align with the pycortex surface or viewing it in 3D in a web browser.

# SECTIONS


* [1. Importing subjects and flatmaps](#import_stuff)
* [2. Alingment and transforms](#align_xforms)
    * 2.1 [Creating transformations inside pycortex (linear only)](#inside_pc)
    * 2.2 [Transforming data outside of pycortex (for using identity transform in pycortex)](#outside_pc) 
* [3. Pycortex interactive web viewer](#pc_webv)




<a id='import_stuff'></a>

## IMPORTING SUBJECTS AND FLATMAPS

ONLY RUN THESE FOR *NEW* SUBJECTS NOT ALREADY IN YOUR PYCORTEX DATABASE \
otherwise subject information could be overwritten



In [2]:
xx_subj = 'subj01'

In [6]:
## IMPORT NEW SUBJECT 
cortex.freesurfer.import_subj(xx_subj)

In [8]:
# IMPORT FSAVERAGE INTO PYCORTEX 
cortex.download_subject('fsaverage')


Downloaded subject fsaverage to /tmp
Extracting subject fsaverage to /home/newpycortex/pycortex_db


In [3]:
## IMPORT FLAT MAP (note: you can ignore overlay error for new subject)
cortex.freesurfer.import_flat(xx_subj, 'full', flat_type = 'fs6')

flatmap vertex locations for this subject, and will result
in deletion of the overlays.svg file and all cached info
for this subject (because flatmaps will fundamentally change).
Proceed? [y]/n: y
b'created by greve on Thu Apr 11 16:23:42 2013\n'
saving to /home/newpycortex/pycortex_db/fsaverage/surfaces/flat_lh.gii
b'created by greve on Thu Apr 11 16:23:51 2013\n'
saving to /home/newpycortex/pycortex_db/fsaverage/surfaces/flat_rh.gii


<a id='align_xforms'></a>

## ALIGNMENT AND TRANSFORMS



<a id='inside_pc'></a>

### CREATING TRANSFORMATIONS INSIDE PYCORTEX (linear only)



In [53]:
output_folder = '/home/breedlov/SDB/nsd_results/transforms/'

In [61]:
## CREATE PYCORTEX AUTO XFORM FOR SINGLE SUBJECT using auto align 

xx_subj = 'subj01'
image2move = '/home/breedlov/SDB/nsd_results/subj01/func1pt8/mean.nii.gz'
new_x_form = 'func1pt8_to_anat0pt8_autoFSbbr'
cortex.align.automatic(xx_subj, new_x_form, image2move, use_fs_bbr=True) # with Freesurfers boundary based registration


Running freesurfer BBR
Success


In [62]:
## CONVERT PYCORTEX XFORM TO FREESURFER XFORM (so you can check alignment with tkregisterfv)

xx_subj = 'subj01'
pc_xform_in = 'func1pt8_to_anat0pt8_autoFSbbr' 
fs_xform_out = f'{output_folder}/{xx_subj}_{pc_xform_in}.dat'
cortex.db.get_xfm(xx_subj, pc_xform_in).to_freesurfer(fs_xform_out, xx_subj)

# use with tkregisterfv --mov yourpath/mean.nii.gz --reg yourpath/subj01_func1pt8_to_anat0pt8_autoFSbbr.dat  --surfs

Wrote:


array([[-0.99785985,  0.0422477 ,  0.04990541,  0.54383688],
       [ 0.04341131, -0.14270093,  0.98881407,  1.71494748],
       [-0.04889662, -0.98886438, -0.1405621 ,  0.47256394],
       [ 0.        ,  0.        ,  0.        ,  1.        ]])

In [9]:
## CREATE PYCORTEX AUTO XFORM FOR MULTIPLE SUBJECTS - for each subj, auto align between input NSD image and anatomical
## WHILE CONVERTING PC XFORM --> FS XFORM

new_x_form = 'func1pt0_to_anat0pt8_autoFSbbr'

for subj in range(1,9):
    xx_subj = 'subj0%s' %(subj)
    print ("\n\ncreating xform for %s..."  %(xx_subj))
    
    image2move = '/home/breedlov/SDB/nsd_results/subj0%s/func1pt0/mean.nii.gz' %(subj)
    cortex.align.automatic(xx_subj, new_x_form, image2move, use_fs_bbr=True)
    
    # create freesurfer xform for checking how auto align did 
    fs_xform = output_folder + xx_subj + new_x_form + '.dat'
    cortex.db.get_xfm(xx_subj, new_x_form).to_freesurfer(fs_xform, xx_subj)
    



creating xform for subj01...
Running freesurfer BBR
Success
Wrote:


creating xform for subj02...
Running freesurfer BBR
Success
Wrote:


creating xform for subj03...
Running freesurfer BBR
Success
Wrote:


creating xform for subj04...
Running freesurfer BBR
Success
Wrote:


creating xform for subj05...
Running freesurfer BBR
Success
Wrote:


creating xform for subj06...
Running freesurfer BBR
Success
Wrote:


creating xform for subj07...
Running freesurfer BBR
Success
Wrote:


creating xform for subj08...
Running freesurfer BBR
Success
Wrote:


[back to top](#back_to_top)

<a id='outside_pc'></a>

### TRANSFORMING DATA OUTSIDE OF PYCORTEX 

USE NSD MAPDATA TO MOVE DATA FROM FUNCT > ANAT \
(can then just use 'identity' transform in pycortex to view this data on the cortical surface)


In [64]:
data_path = '/home/breedlov/SDB/nsd_results/'
nsd_dir = nsd_datalocation()
print (nsd_dir)

/home/breedlov/SDA/cmrr/nsd/nsddata


In [67]:
# MAP FUNCT DATA/RESULTS TO ANAT SPACE FOR SINGLE SUBJ

subjix = 1
sourcedata = '/home/breedlov/SDB/nsd_results/subj01/func1pt8/mean.nii.gz'
sourcespace = 'func1pt8'
targetspace = 'anat0pt8'
interpmethod = 'nearest'


sd_shortname = sourcedata.split('/')[-1].split('.')[0]
outputfile=f'{data_path}subj0{subjix}/{targetspace}/{sd_shortname}_in_{targetspace}_{interpmethod}.nii'

targetdata = nsd_mapdata(
    subjix,
    sourcespace,
    targetspace,
    sourcedata,
    interptype=interpmethod,
    badval=0,
    outputfile=outputfile)


In [65]:
# MAP FUNCT DATA/RESULTS TO ANAT SPACE FOR MULTIPLE SUBJS
# loop through subjects
# look for nifti files with strings from [data_list] in the name
# create a copy that is transformed into the desired space

data_list = ['val_cc', 'rf_ang', 'rf_ecc', 'rf_size', 'layer_spread', 'layer_tuning', 'mean']

sourcespace = 'func1pt8'
targetspace = 'anat0pt8'
interpmethod = 'nearest'

for subjix in range(1,9):
    print ("\n\n****************  subj0%s  ****************" %(subjix))
    
    for d in data_list:
        
        file_name = glob.glob(f'{data_path}subj0{subjix}/{sourcespace}/{d}*.nii*')
        
        if file_name==[]:
            print('\nWarning: there is no *%s* data file for subj0%s' %(d, subjix))
        elif len(file_name) > 1:
            print('\nWarning: multiple *%s* data files for subj0%s, nothing was transformed' %(d, subjix))
        else:
            
            sourcedata = file_name[0]
            sd_shortname = sourcedata.split('/')[-1].split('.')[0]
            
            if not os.path.isdir(f'{data_path}subj0{subjix}/{targetspace}'):
                os.mkdir(f'{data_path}subj0{subjix}/{targetspace}')
                
            outputfile=f'{data_path}subj0{subjix}/{targetspace}/{sd_shortname}_in_{targetspace}_{interpmethod}.nii'
            print('\nsource data:', sourcedata)
            print('moving source data to %s space....' %(targetspace))
            
            targetdata = nsd_mapdata(
                subjix,
                sourcespace,
                targetspace,
                sourcedata,
                interptype=interpmethod,
                badval=0,
                outputfile=outputfile)
            
            print ('output data:', outputfile)




****************  subj01  ****************

source data: /home/breedlov/SDB/nsd_results/subj01/func1pt8/val_cc_Feb-05-2020_1609.nii
moving source data to anat0pt8 space....
output data: /home/breedlov/SDB/nsd_results/subj01/anat0pt8/val_cc_Feb-05-2020_1609_in_anat0pt8_nearest.nii

source data: /home/breedlov/SDB/nsd_results/subj01/func1pt8/rf_ang_Feb-05-2020_1609.nii
moving source data to anat0pt8 space....
output data: /home/breedlov/SDB/nsd_results/subj01/anat0pt8/rf_ang_Feb-05-2020_1609_in_anat0pt8_nearest.nii

source data: /home/breedlov/SDB/nsd_results/subj01/func1pt8/rf_ecc_Feb-05-2020_1609.nii
moving source data to anat0pt8 space....
output data: /home/breedlov/SDB/nsd_results/subj01/anat0pt8/rf_ecc_Feb-05-2020_1609_in_anat0pt8_nearest.nii

source data: /home/breedlov/SDB/nsd_results/subj01/func1pt8/rf_size_Feb-05-2020_1609.nii
moving source data to anat0pt8 space....
output data: /home/breedlov/SDB/nsd_results/subj01/anat0pt8/rf_size_Feb-05-2020_1609_in_anat0pt8_nearest.nii



output data: /home/breedlov/SDB/nsd_results/subj05/anat0pt8/layer_tuning_Feb-06-2020_2152_in_anat0pt8_nearest.nii

source data: /home/breedlov/SDB/nsd_results/subj05/func1pt8/mean.nii.gz
moving source data to anat0pt8 space....
output data: /home/breedlov/SDB/nsd_results/subj05/anat0pt8/mean_in_anat0pt8_nearest.nii


****************  subj06  ****************

source data: /home/breedlov/SDB/nsd_results/subj06/func1pt8/val_cc_Feb-09-2020_1605.nii
moving source data to anat0pt8 space....
output data: /home/breedlov/SDB/nsd_results/subj06/anat0pt8/val_cc_Feb-09-2020_1605_in_anat0pt8_nearest.nii

source data: /home/breedlov/SDB/nsd_results/subj06/func1pt8/rf_ang_Feb-09-2020_1605.nii
moving source data to anat0pt8 space....
output data: /home/breedlov/SDB/nsd_results/subj06/anat0pt8/rf_ang_Feb-09-2020_1605_in_anat0pt8_nearest.nii

source data: /home/breedlov/SDB/nsd_results/subj06/func1pt8/rf_ecc_Feb-09-2020_1605.nii
moving source data to anat0pt8 space....
output data: /home/breedlov/SDB/

[back to top](#back_to_top)

<a id='pc_webv'></a>

## PYCORTEX INTERACTIVE WEBVIEWER



In [60]:
## VIEW DATA ON SURFACE IN BROWSER WINDOW

xx_subj = 'subj01'
data_file = '/nsd_results/subj01/anat0pt8/val_cc_Feb-05-2020_1609_in_anat0pt8_nearest.nii' 
x_form = 'identity' #'func1pt8_to_anat0pt8_autoFSbbr'

data = nibabel.load(data_file) 
arr = data.get_fdata().T.astype(float) 

if x_form == 'identity':   #data that aligns to the subj anatomical must first be tipped forward 90deg then flipped left/right
    arr = np.flip((np.rot90(arr, k=3)),2)
    
vol = cortex.Volume(arr, xx_subj, x_form)
h = cortex.webgl.show(vol)

Started server on port 25766
Stopping server
Stopping server
Stopping server
Stopping server


In [68]:
## VIEW SUBJECTS OWN ANATOMICAL AS DATA USING BUILT IN IDENTITY TRANSFORM

subj_xx = 'subj02'
the_anat = cortex.db.get_anat(subj_xx, type='raw')
arr = the_anat.get_fdata().T
vol = cortex.Volume(arr, subj_xx, 'identity', recache=True)
h = cortex.webgl.show(vol)

Started server on port 8775


In [6]:
## VIEW 2D DATA ON SURFACE 

xx_subj = 'subj01'
x_form = 'identity' #'func1pt8_to_anat0pt8_autoFSbbr'

data_file1 = '/nsd_results/subj01/anat0pt8/rf_ang_Feb-05-2020_1609_in_anat0pt8_nearest.nii' 
data_file2 = '/nsd_results/subj01/anat0pt8/val_cc_Feb-05-2020_1609_in_anat0pt8_nearest.nii'

data1 = nibabel.load(data_file1) 
arr1 = data1.get_fdata().T.astype(float) 
data2 = nibabel.load(data_file2) 
arr2 = data2.get_fdata().T.astype(float) 

if x_form == 'identity':   #identity isn't quite true, data that aligns to the subj anatomical must first be tipped forward 90deg then flipped left/right
    arr1 = np.flip((np.rot90(arr1, k=3)),2)
    arr2 = np.flip((np.rot90(arr2, k=3)),2)

vol2D = cortex.Volume2D(arr1, arr2, xx_subj, x_form)
h = cortex.webgl.show(vol2D)

Started server on port 56964


In [69]:
## VIEW MULTIPLE DATASETS IN SINGLE SESSION
## note: toggle between them using expandable box in the top left corner; you can also combine two datasets in the
## the webviewer by clicking and dragging mouse over 2 of these labels (this creates a 2D dataset)

xx_subj = 'subj01'
x_form = 'identity' #'func1pt8_to_anat0pt8_autoFSbbr'

data_file1 = '/nsd_results/subj01/anat0pt8/visB_v_model_cc_Feb-05-2020_1609_in_anat0pt8_nearest.nii' 
data_file2 = '/nsd_results/subj01/anat0pt8/imgB12_v_model_cc_Feb-05-2020_1609_in_anat0pt8_nearest.nii'


data1 = nibabel.load(data_file1) 
arr1 = data1.get_fdata().T.astype(float) 
data2 = nibabel.load(data_file2) 
arr2 = data2.get_fdata().T.astype(float) 

if x_form == 'identity':   #data that aligns to the subj anatomical must first be tipped forward 90deg then flipped left/right
    arr1 = np.flip((np.rot90(arr1, k=3)),2)
    arr2 = np.flip((np.rot90(arr2, k=3)),2)

vol1 = cortex.Volume(arr1, xx_subj, x_form)    
vol2 = cortex.Volume(arr2, xx_subj, x_form) #just repeat the above steps for more than 2 vols  

volumes = {
	'visB_v_model_cc': vol1,
	'imgB_v_model_cc': vol2
}

h = cortex.webgl.show(data=volumes)

Started server on port 62992


In [5]:
# SAVE CURRENTLY DISPLAYED VIEW TO A .png IMAGE FILE 

h.getImage('/nsd_results/test.png', size=(1920,1080))

[{}]

[back to top](#back_to_top)